In [36]:
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

In [14]:
tempIemisions = pd.read_csv("DataSetFinal.csv", index_col=0)
dummiesCountry=pd.get_dummies(tempIemisions['Country'],drop_first=1)
tempIemisions=pd.concat([tempIemisions,dummiesCountry],axis=1)
tempIemisions=tempIemisions.drop(['Country'], axis=1)

#fem X totes les columnes menys temp i y nomes temp el axis=1 es per indicar
#que es una label de les columnes
X = tempIemisions.drop('AverageTemperature', axis=1)
y = tempIemisions['AverageTemperature']

#dividim el dataSet en test 20% i entrenament 80%
#el random_state es la llavor que s'utilitzarà per randomitzar 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)


31


In [44]:
model = Sequential()
model.add(Dense(55, input_dim=X_train.shape[1], activation='relu', kernel_initializer='normal'))
model.add(Dense(31, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mae', optimizer='adam', metrics=['mae'])
# Fit the model
model.fit(X_train, y_train, epochs=2400,validation_split = 0.2, batch_size=84 ,verbose=0)

y_pred = model.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))

Mean Absolute Error: 3.842551104260962


In [45]:
#millor de moment 
model = Sequential()
model.add(Dense(55, input_dim=X_train.shape[1], activation='relu', kernel_initializer='normal'))
model.add(Dense(31, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mae', optimizer='adam', metrics=['mae'])
# Fit the model
model.fit(X_train, y_train, epochs=2200,validation_split = 0.2, batch_size=84 ,verbose=0)

y_pred = model.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))

Mean Absolute Error: 3.096557720332718


In [41]:
# Function to create model, required for KerasRegressor
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(55, input_dim=X_train.shape[1], activation='relu', kernel_initializer='normal'))
    model.add(Dense(31, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='linear'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasRegressor(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [64, 60, 62, 70, 80]
epochs = [2000, 2200, 2400]
validation_split = [0.2, 0.22, 0.25, 0.3]
param_grid = dict(batch_size=batch_size, epochs=epochs, validation_split=validation_split)
scoring = 'neg_mean_absolute_error'
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, n_jobs=-1, verbose = 2)
grid.fit(X_train, y_train)
print('Millor combinacio:', grid.best_params_)
y_pred = grid.predict(X_test)

# summarize results
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

C:\Users\Miquel\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 27.6min
C:\Users\Miquel\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 116.8min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed: 132.2min finished


Millor combinacio: {'batch_size': 64, 'epochs': 2400, 'validation_split': 0.2}
Mean Absolute Error: 4083.516223041373
Mean Squared Error: 46504958.23685385
Root Mean Squared Error: 6819.454394367181
